In [10]:
import pandas as pd
import numpy as np
import json
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

country = "Lesotho"

# Load datasets
schools_df = pd.read_csv('dataset/school_data.csv')
towers_df = pd.read_csv('dataset/lesotho.csv')
demographic_df = pd.read_csv('dataset/demographic_data_lesotho_v2.csv')
elevation_df = pd.read_csv('dataset/satellite_data_lesotho_v2.csv')
spectrum_df = pd.read_csv('dataset/tv_spectrum_lesotho.csv')

# Extract relevant columns
schools = schools_df[['School Name', 'Longitude', 'Latitude']].dropna()
towers = towers_df[['lon', 'lat', 'range']].dropna()
spectrum = spectrum_df[['latitude', 'longitude', 'available_channels', 'interference_level']].dropna()
elevation = elevation_df[['latitude', 'longitude', 'elevation']]
demographics = demographic_df[['region', 'population', 'density']]

# Define estimated TVWS coverage range (km)
TVWS_RANGE_URBAN = 10
TVWS_RANGE_RURAL = 30

def is_covered_by_tower(school, towers):
    """Check if a school is within the range of an existing tower."""
    for _, tower in towers.iterrows():
        distance = geodesic((school['Latitude'], school['Longitude']), (tower['lat'], tower['lon'])).km
        if distance <= tower['range'] / 1000:  # Convert meters to km
            return True
    return False

# Filter out schools already covered by existing towers
schools = schools[~schools.apply(lambda x: is_covered_by_tower(x, towers), axis=1)]

def cluster_schools(df, method='dbscan'):
    """Dynamically determine clustering radius based on school density."""
    num_schools = len(df)
    eps = max(2, min(10, num_schools / 50))  # Adjust eps dynamically based on density
    min_samples = max(2, min(5, num_schools // 100))
    coords = df[['Latitude', 'Longitude']].values
    
    if method == 'dbscan':
        cluster = DBSCAN(eps=eps/6371, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    elif method == 'agglomerative':
        cluster = AgglomerativeClustering(n_clusters=None, distance_threshold=eps).fit(coords)
    elif method == 'gmm':
        gmm = GaussianMixture(n_components=min(10, len(df)//5), random_state=42)
        cluster = gmm.fit_predict(coords)
    else:
        cluster = KMeans(n_clusters=min(10, len(df)//5), random_state=42).fit(coords)
    
    df['Cluster'] = cluster if isinstance(cluster, np.ndarray) else cluster.labels_
    return df

# Cluster the schools dynamically with Agglomerative Clustering or GMM for flexibility
schools_clustered = cluster_schools(schools, method='agglomerative')

def find_tvws_locations_and_extract_schools(df, schools_df):
    clusters = df.groupby('Cluster')
    affected_schools_set = set()  # Use a set to avoid duplicate schools

    # Build a Random Forest regressor to predict the best location
    X = df[['Latitude', 'Longitude']]
    y = np.random.rand(len(df))  # This can be improved with real metrics, e.g., potential coverage score
    
    # Train-test split for regression model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)
    print(f"MAE on test set: {mean_absolute_error(y_test, y_pred)}")
    
    for cluster_id, group in clusters:
        if cluster_id == -1:
            continue
        
        lat_mean, lon_mean = group['Latitude'].mean(), group['Longitude'].mean()
        
        # Find the closest available spectrum location
        spectrum_nearby = spectrum.iloc[((spectrum['latitude'] - lat_mean)**2 + (spectrum['longitude'] - lon_mean)**2).idxmin()]
        available_channels = json.loads(spectrum_nearby['available_channels'].replace("'", "\"")) if isinstance(spectrum_nearby['available_channels'], str) else spectrum_nearby['available_channels']
        
        # Find elevation and demographic data
        elevation_nearby = elevation.iloc[((elevation['latitude'] - lat_mean)**2 + (elevation['longitude'] - lon_mean)**2).idxmin()]
        demographic_data = demographics.iloc[0]  # Assume one region for now
        
        # Determine coverage range based on population density
        tvws_range = TVWS_RANGE_URBAN if demographic_data['density'] > 100 else TVWS_RANGE_RURAL
        tvws_id=f"{country}_tvws_{int(cluster_id)}"
        
        # Find schools within the TVWS coverage range
        affected_schools = group[
            group.apply(lambda x: geodesic((lat_mean, lon_mean), (x['Latitude'], x['Longitude'])).km <= tvws_range, axis=1)
        ]
        
        # Add the affected schools to the set to ensure uniqueness
        for school_name in affected_schools['School Name']:
            affected_schools_set.add(school_name)
    
    # Now filter the original schools_df to include only the affected schools
    affected_schools_df = schools_df[schools_df['School Name'].isin(affected_schools_set)]

    # Save the affected schools to a new CSV file
    affected_schools_df.to_csv('affected_schools_only.csv', index=False)

    return affected_schools_df

# Determine affected schools and save the new CSV
affected_schools_df = find_tvws_locations_and_extract_schools(schools_clustered, schools_df)

# Generate TVWS GeoJSON data for the affected schools (without changing the previous GeoJSON structure)
tvws_geojson = {
    'type': 'FeatureCollection',
    'features': []
}

# Populate TVWS GeoJSON with existing logic
for cluster_id, group in schools_clustered.groupby('Cluster'):
    if cluster_id == -1:
        continue
    
    lat_mean, lon_mean = group['Latitude'].mean(), group['Longitude'].mean()
    
    # Find the closest available spectrum location
    spectrum_nearby = spectrum.iloc[((spectrum['latitude'] - lat_mean)**2 + (spectrum['longitude'] - lon_mean)**2).idxmin()]
    available_channels = json.loads(spectrum_nearby['available_channels'].replace("'", "\"")) if isinstance(spectrum_nearby['available_channels'], str) else spectrum_nearby['available_channels']
    
    # Find elevation and demographic data
    elevation_nearby = elevation.iloc[((elevation['latitude'] - lat_mean)**2 + (elevation['longitude'] - lon_mean)**2).idxmin()]
    demographic_data = demographics.iloc[0]  # Assume one region for now
    
    # Determine coverage range based on population density
    tvws_range = TVWS_RANGE_URBAN if demographic_data['density'] > 100 else TVWS_RANGE_RURAL
    tvws_id = f"{country}_tvws_{int(cluster_id)}"
    
    # Add TVWS data to GeoJSON
    tvws_geojson['features'].append({
        'type': 'Feature',
        'properties': {
            'Cluster': tvws_id,
            'Available Channels': available_channels,
            'Interference': spectrum_nearby['interference_level'],
            'Elevation': float(elevation_nearby['elevation']),
            'Population': int(demographic_data['population']),
            'Density': float(demographic_data['density']),
            'TVWS Range (km)': tvws_range
        },
        'geometry': {
            'type': 'Point',
            'coordinates': [float(lon_mean), float(lat_mean)]
        }
    })

# Save the TVWS station locations to GeoJSON
with open('tvws_stations.geojson', 'w') as f:
    json.dump(tvws_geojson, f, indent=4)

print("TVWS station locations calculated and saved as 'tvws_stations.geojson'")
print("Schools affected by TVWS stations saved as 'affected_schools_only.csv'")


MAE on test set: 0.25833083250275907
TVWS station locations calculated and saved as 'tvws_stations.geojson'
Schools affected by TVWS stations saved as 'affected_schools_only.csv'
